In [ ]:
El conjunto de datos incluye más de 50 características que representan los resultados de pacientes y hospitales. Se extrajo información de la base de datos para los casos que cumplieron con los siguientes criterios:

a) Se trata de un ingreso hospitalario.

b) Se trata de un caso diabético, es decir, uno durante el cual se registró cualquier tipo de diabetes en el sistema como diagnóstico.

c) La duración de la estancia fue de al menos 1 día y de un máximo de 14 días.

d) Se realizaron análisis de laboratorio durante el encuentro.

e) Se administraron medicamentos durante el encuentro.

Los datos contienen atributos como el número de paciente, raza, sexo, edad, tipo de ingreso, tiempo de hospitalización, especialidad médica del médico que lo admitió, número de análisis de laboratorio realizados, resultado de la prueba de HbA1c, diagnóstico, número de medicamentos, medicamentos para la diabetes, número de visitas ambulatorias, hospitalarias y de urgencias en el año anterior a la hospitalización, etc.

### Puntos a evaluar:

La actividad se conforma por:

    Encontró 10 hallazgos relevantes para el problema descripto anteriormente mediante el uso de medidas estadísticas y gráficos. (5pts)
    Aplicó las siguientes técnicas de preprocesamiento de datos:
        Limpieza
            Manejo de datos faltantes (1pt)
            Manejo de casos atípicos (1pt)
            Datos duplicados (1pt)
            Conversión de tipos (1pt)
            Limpieza de texto (1pt)
            Renombrar columnas (1pt)
            Filtrar filas (1pt)
            Documenta y justifica porque se aplico cada técnica (3)
        Integración de datos (10pts)
            Agregación/fusión o concatenación


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dict = pd.read_excel('dictionary_.xlsx')
ids_map_adm_typ = pd.read_excel('IDS_mapping_exc.xlsx', sheet_name='admission_type_id')
ids_map_dis = pd.read_excel('IDS_mapping_exc.xlsx', sheet_name='discharge_id')
ids_map_adm_src = pd.read_excel('IDS_mapping_exc.xlsx', sheet_name='admission_source_id')
df = pd.read_csv('diabetic_data.csv')

ids_map_adm_src

## Dropeo de columnas
Algunas columnas van a ser dropeadas, a continuacion cuales y su justificacion:

* encounter_id: Como es unico, no nos puede servir para sacar datos
* weight: demasiados datos faltantes
* payer code: no hay suficiente informacion

### Medicinas:

Las siguientes se van a mantener. De acuerdo a una inteligencia artifical, las medicinas que podrian indicar un futuro reingreso son:

'insulin', 'glyburide', 'glipizide', 'glimepiride', 'tolbutamide', 'chlorpropamide', 'tolazamide', 'metformin', 'pioglitazone', 'rosiglitazone',
Las demas no seran tomadas en cuenta

Pese a su importancia, max_glu_serum y A1Cresult se eliminaran puesto que la gran mayoria de los datos son faltantes, hablando de   > 80%
## Por que no se eliminan ...
*   patient_nbr: Puesto que pueden tener varias visitas con el mismo id de paciente, pues es el mismo

In [ ]:
'''
conteo = df['patient_nbr'].value_counts()
duplicados = conteo[conteo > 1]
print(duplicados)
'''
columnas_drop = ['encounter_id','weight','payer_code','repaglinide', 'nateglinide', 'acetohexamide', 'acarbose', 'miglitol', 'troglitazone', 'examide', 'citoglipton', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'max_glu_serum','A1Cresult']

df = df.drop(columns = columnas_drop)


# Renombramiento de columnas

In [ ]:
 #se deja mas claro de esta forma
df.rename(columns={'time_in_hospital': 'days_stayed'}, inplace=True)
df.rename(columns={'number_outpatient': 'consults_only_prev'}, inplace=True)
df.rename(columns={'number_emergency': 'amount_emergencies'}, inplace=True)
df.rename(columns={'number_inpatient': 'consult_stayed_prev'}, inplace=True)

# Outliers
Los outliers pueden proporcionar datos que llegan a afectar a lo que es lo general, por lo cual se opta por eliminarlos

In [ ]:
Q1 = df["num_lab_procedures"].quantile(0.25)
Q3 = df["num_lab_procedures"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
# Identificar valores atípicos
outliers = df[(df["num_lab_procedures"] < lower_bound) | (df["num_lab_procedures"] > upper_bound)]
# print(f"\nValores atípicos en '{"num_lab_procedures"}':\n", outliers["num_lab_procedures"])
# Eliminar valores atípicos (o podrías limitarlos, transformarlos, etc.)
df = df[(df["num_lab_procedures"] >= lower_bound) & (df["num_lab_procedures"] <= upper_bound)]

Q1 = df["num_procedures"].quantile(0.25)
Q3 = df["num_procedures"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
# Identificar valores atípicos
outliers = df[(df["num_procedures"] < lower_bound) | (df["num_procedures"] > upper_bound)]
# print(f"\nValores atípicos en '{"num_lab_procedures"}':\n", outliers["num_lab_procedures"])
# Eliminar valores atípicos (o podrías limitarlos, transformarlos, etc.)
df = df[(df["num_procedures"] >= lower_bound) & (df["num_procedures"] <= upper_bound)]

In [ ]:
# Mantiene solo las filas de < 30, pues son las que nos pueden brindar los patrones
df_menor30 = df[df['readmitted'] == '<30']
df_menor30
# Al querer datos generales, los outliers pueden

In [ ]:
# Integracion de Datos
df = df.merge(ids_map_adm_typ, on='admission_type_id', how='inner')
df = df.merge(ids_map_dis, on='discharge_disposition_id', how='inner')
df = df.merge(ids_map_adm_src, on='admission_source_id', how='inner')
df

In [ ]:
# Graficas y hallazgos
df_m30_age = df_menor30['age'].value_counts()

df_m30_age.sort_index().plot(kind='bar', color='skyblue', edgecolor='black')
plt.xlabel('Age')
plt.ylabel('Count')
plt.title('Distribucion de Edad')
plt.xticks(rotation=45)

## Hallazgo 1.
Entre mas edad tiene el paciente, es mas probable que regrese dentro de 30 dias para su chequeo. Las edades 80+ empiezan a bajar debido a que el grupo de poblacion disminuye (hay menos de esa edad).

In [ ]:
# Graficas y hallazgos
df_m30_race = df_menor30['race'].value_counts()

df_m30_race.plot(kind='pie', autopct='%1.1f%%', startangle=90, figsize=(6,6))

plt.ylabel('')  # Hide the default y-label
plt.title('Distribución de Raza')

plt.show()

### HALLAZGO 2

Los de raza caucasian son mucho mas probables de regresar al hospital

In [ ]:
# Graficas y hallazgos
df_m30_race = df_menor30['gender'].value_counts()

df_m30_race.plot(kind='pie', autopct='%1.1f%%', startangle=90, figsize=(6,6))

plt.ylabel('')  # Hide the default y-label
plt.title('Distribución de Genero')

plt.show()

### Hallazgo 3
La distribucion es casi la misma, aunque es mas probable que las mujeres regresen dentro del periodo de tiempo indicado

In [ ]:
plt.figure(figsize=(6, 4))  # Set figure size
plt.boxplot(df_menor30['days_stayed'], patch_artist=True)

# Add labels and title
plt.ylabel('Days Stayed')
plt.title('Boxplot de Días de Estancia para Pacientes <30')

# Show the plot
plt.show()

### Hallazgo 4
La mediana de dias que se quedan en el hospital, y regresan, es de 4. Puede ser que no se termine de tratar

In [ ]:
plt.figure(figsize=(6, 4))  # Set figure size
plt.boxplot(df_menor30['num_lab_procedures'], patch_artist=True)

# Add labels and title
plt.ylabel('Num procedimientos')
plt.title('Distribucion de procedimients hechos en la estancia')

# Show the plot
plt.show()

### Hallazgo 5:
La mediana de los que regresan se encuentran como 50, por lo cual si se llega a esa cantidad de procedimientos, si tiene le riesgo de regresar dentro de 30 dias

In [ ]:
# Graficas y hallazgos
df_m30_age = df_menor30['num_medications'].value_counts()

df_m30_age.sort_index().plot(kind='bar', color='skyblue', edgecolor='black')
plt.xlabel('Num medications')
plt.ylabel('Count')
plt.title('Distribucion de Medicamentos aplicados')
plt.xticks(rotation=90)

Hallazgo 6

Entre